In [30]:
!wget -nc https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

File ‘input.txt’ already there; not retrieving.



In [3]:
#read it to inspect it
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [4]:
print("length of the number of chracters: " + str(len(text)))

length of the number of chracters: 1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# Unique characters of the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# Create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
print(stoi)
print(itos)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [8]:
# Enconder: take a string, output the list of integers assigned to each.
encode = lambda s: [stoi[c] for c in s]
# Decoder: Take a list  of integers, output a string
decode = lambda s: [itos[c] for c in s] 

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd']


In [9]:
# Transform it into a torch of integers to be used in Deep Learning
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/williehernandez/Documents/github/buildgpt_from_zero/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/williehernandez/Documents/github/buildgpt_from_zero/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/williehernandez/Documents/github/buildgpt_from_ze

torch.Size([1115394]) torch.int64


In [10]:
# Split de data into training and evaluation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8 #Basically the batches for training fo the data

In [12]:
# Intuition: We want that our algorithm sees a sequence of integers from 1 to 8 (block size) and has the abillity to predict the next character.
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

# Note: The transformer should be comfortable being context of size 1 and of size 8.

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [13]:
# Process multiple batches at the same time to make use of the GPU
torch.manual_seed(1337)
batch_size = 4 # number of independent sequences will we process in parallel.
block_size = 8 # maximum number of chracters in the same block

In [14]:
def get_batch(split):
    """
    Generate a batch of input and target sequences for training or validation.

    This function randomly samples `batch_size` starting positions from either 
    the training or validation dataset and returns two tensors:
      - x (inputs): sequences of length `block_size`
      - y (targets): the same sequences shifted one position to the right

    Parameters
    ----------
    split : str
        The dataset to draw from. Must be either:
          - "train" : use the global variable `train_data`
          - "val"   : use the global variable `val_data`

    Returns
    -------
    x : torch.Tensor, shape (batch_size, block_size), dtype=torch.long
        The input sequences, each of length `block_size`. 
        Each row corresponds to one sequence sampled from the dataset.

    y : torch.Tensor, shape (batch_size, block_size), dtype=torch.long
        The target sequences, each also of length `block_size`.
        For each row, `y` is the same as `x` but shifted by one character/token.
        (If x is [c1, c2, c3, ...], then y is [c2, c3, c4, ...])

    Notes
    -----
    - `ix` contains the random starting indices for each sequence.
    - The slicing ensures that `y` is the "next-token prediction target" 
      for `x`, which is the standard setup for language modeling.
    - Both x and y are stacked into tensors so they can be processed in 
      parallel by the model (GPU-friendly).
    """
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [15]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


**B (Batch size)** = number of independent sequences processed in parallel.

**T (Time / Context length)** = max number of tokens per sequence.

**C (Channels / Classes)** = vocabulary size.

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    """
    BigramLanguageModel

    This simple model learns to predict the next token using *only the current token*
    (no context beyond one step = "bigram").

    Tensor dimensions:
        B = batch size
            → number of independent sequences processed in parallel.

        T = time (context length)
            → maximum number of tokens per sequence.
              Each row has T tokens, so total tokens in a batch = B * T.
        C = vocab size
            → number of output classes (the logits for each next token).

    Flow:
        idx: (B, T) integers [0, vocab_size-1]
        targets: (B, T)
        logits: (B, T, C) → reshaped to (B*T, C)
        loss: scalar cross-entropy over all B*T predictions
    """

    def __init__(self, vocab_size: int):
        super().__init__()
        # Embedding table: each input token maps to a row of length vocab_size.
        # Shape: (vocab_size, vocab_size)
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor):
        """
        Forward pass.

        Parameters
        ----------
        inputs : torch.Tensor
            Shape (B, T) with token ids.
        targets : torch.Tensor
            Shape (B, T) with ground-truth next token ids.

        Returns
        -------
        logits : torch.Tensor
            Shape (B*T, C). Logits for every token position.
        loss : torch.Tensor
            Scalar mean cross-entropy loss.
        """
        if targets is None:
            loss = None
        else:
            # (B, T, C): each token index is mapped to a vocab-sized vector
            logits_3d = self.token_embedding_table(inputs)

            B, T, C = logits_3d.shape
            # Flatten batch and time so CE sees (N, C) with N=B*T
            logits = logits_3d.view(B * T, C)
            flat_targets = targets.view(B * T)

            loss = F.cross_entropy(logits, flat_targets)
        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        """
        Autoregressive text generation.

        Starting from an initial sequence of token indices, repeatedly sample
        the next token using the model’s bigram predictions and append it.

        Parameters
        ----------
        idx : torch.Tensor
            Shape (B, T) with initial token ids (the "prompt").
        max_new_tokens : int
            Number of tokens to generate beyond the initial prompt.

        Returns
        -------
        idx : torch.Tensor
            Shape (B, T + max_new_tokens). The original prompt followed by
            the newly generated tokens.
        """
        for _ in range(max_new_tokens):
            # Forward pass: (B, T, C) logits for all positions
            logits_3d = self.token_embedding_table(idx)

            # Focus only on the last time step
            # Shape: (B, C)
            logits_last = logits_3d[:, -1, :]

            # Convert logits to probabilities via softmax
            probs = F.softmax(logits_last, dim=-1)  # (B, C)

            # Sample next token from the probability distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # Append new token to the sequence along the time dimension
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

# Perplexity = exp(loss)
perplexity = torch.exp(loss)

print("Logits shape:", logits.shape)     # (B*T, C) → (32, 65)
print("Loss:", loss.item())              # scalar
print("Perplexity:", perplexity.item())  # scalar

Logits shape: torch.Size([32, 65])
Loss: 4.878634929656982
Perplexity: 131.4510955810547


In [17]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(''.join(decode(m.generate(idx, max_new_tokens=100)[0].tolist())))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [18]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [19]:
batch_size = 32
for steps in range(100000):

  # Sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.404492139816284


In [20]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(''.join(decode(m.generate(idx, max_new_tokens=100)[0].tolist())))


Bere ak hongacl GELusey wssisean thi's.
When, w t med yfthon oklichay ol, I INI' dede blais cicen m 


# Mathematical simple option of Self-Attention Mechanism

In [21]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # Batch, Tokens, Vocab Size
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# We want x[b,t] = mean_{i<=t} x[b, i]
# The attention here is that what is the average of the same vocab character for
# all the precedent characters.
xbow = torch.zeros(B, T, C, device=x.device, dtype=x.dtype)
for b in range(B):
    for t in range(T):
        xbow[b, t] = x[b, :t+1].mean(dim=0)

In [23]:
# In order not to repeat using for, using Matrix multiplication
wei = torch.tril(torch.ones(T, T, device=x.device, dtype=x.dtype))
wei = wei / wei.sum(dim=1, keepdim=True)   # row-normalize
xbow2 = wei @ x

In [25]:
xbow[0], xbow2[0] # They are the same

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)

B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

head_size = 16
key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)                  # (B, T, H)
q = query(x)                # (B, T, H)
v = value(x)                # (B, T, H)

# scaled dot-product attention scores
wei = q @ k.transpose(-2, -1) / (head_size ** 0.5)  # (B, T, T)

# causal mask (lower triangular)
tril = torch.tril(torch.ones(T, T, device=wei.device)).bool()
wei = wei.masked_fill(~tril, float('-inf'))

# normalize to probabilities
wei = F.softmax(wei, dim=-1)  # (B, T, T)

# weighted sum of values
out = wei @ v  # (B, T, H)

out.shape  # -> torch.Size([4, 8, 16])


torch.Size([4, 8, 16])

# Lessons from above

- **Self-attention:** can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other (each batch works independently from other)
- 

In [26]:
7+7

14